In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/GaParmar/img2img-turbo.git

Cloning into 'img2img-turbo'...
remote: Enumerating objects: 255, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 255 (delta 88), reused 57 (delta 56), pack-reused 139 (from 2)
Receiving objects: 100% (255/255), 31.91 MiB | 15.85 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [ ]:
import os
os.chdir('img2img-turbo/')

In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-ofghwle3/clip_a747003176b74a149f881dcf67c14376
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-ofghwle3/clip_a747003176b74a149f881dcf67c14376
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/

In [ ]:
# Pointing to dataset loc
import os, pathlib, shutil

driveFolderPath = "/content/drive/MyDrive/training_data/img2img_v1_TE23D_ICCV"  # folder matching required training folder structure (train_A, train_B, test_A, test_B + fixed prompts)
datasetName = "sat2uav_1"

repoRootFolder = "/content/img2img-turbo"
dataDstFolder = f"{repoRootFolder}/data/{datasetName}"
os.makedirs(f"{repoRootFolder}/data", exist_ok=True)

# Since dataset is already organised as specified, symlink easier to implement.
if os.path.exists(dataDstFolder):
    shutil.rmtree(dataDstFolder)
os.symlink(driveFolderPath, dataDstFolder, target_is_directory=True)

# Double checking file listing
for split in ("train_A","train_B","test_A","test_B"):
    print(split, "=>", len(list(pathlib.Path(dataDstFolder, split).glob("*"))), "files")


train_A => 1065 files
train_B => 1073 files
test_A => 117 files
test_B => 119 files


In [ ]:
# Creating/modifying the required fixed prompts dataset (as per img2img training doc).
open(f"{dataDstFolder}/fixed_prompt_a.txt","w").write("overhead satellite scene")
open(f"{dataDstFolder}/fixed_prompt_b.txt","w").write("overhead uav scene")

In [ ]:
!pwd && ls

/content/img2img-turbo
assets	environment.yaml	LICENSE		  scripts
data	gradio_canny2image.py	README.md	  src
docs	gradio_sketch2image.py	requirements.txt  style.css


In [ ]:
!env

In [ ]:
 !nvidia-smi

Fri Oct 31 14:09:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!accelerate config

--------------------------------------------------------------------------------In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)
0
This machine
--------------------------------------------------------------------------------Which type of machine are you using?
Please input a choice index (starting from 0), and press enter
 ➔  No distributed training
    multi-CPU
    multi-XPU
    multi-HPU
    multi-GPU
    multi-NPU
    multi-MLU
    multi-SDAA
    multi-MUSA
    TPU
0
No distributed training
Do you want to run your training on CPU only (even if a GPU / Apple Silicon / Ascend NPU device is available)? [yes/NO]:NO
Do you wish to optimize your script with torch dynamo?[yes/NO]:NO
Do you want to use DeepSpeed? [yes/NO]: NO
What GPU(s) (by id) should be used for training on this machine as a comma-separated list? [all]:0
Would you like to enable numa efficiency? (Currently only support

In [ ]:
%%bash
export NCCL_P2P_DISABLE=1
export TF_CPP_MIN_LOG_LEVEL=3
export WANDB_MODE=offline            # making wandb offline and disabling because of errors due to authenticating login
export WANDB_PROJECT=horse2zebra
export WANDB_DISABLE_CODE=true
accelerate launch --main_process_port 29501 src/train_cyclegan_turbo.py \
    --pretrained_model_name_or_path="stabilityai/sd-turbo" \
    --output_dir="/content/drive/MyDrive/outputs/sat2uav_v1" \
    --dataset_folder "/content/drive/MyDrive/training_data/img2img_v1_TE23D_ICCV" \
    --train_img_prep "resize_286_randomcrop_256x256_hflip" --val_img_prep "no_resize" \
    --learning_rate="1e-5" --max_train_steps=3200 \
    --train_batch_size=1 --gradient_accumulation_steps=1 \
    --report_to "wandb" --tracker_project_name "sat2uav_full_512_a100" \
    --enable_xformers_memory_efficient_attention --validation_steps 50 \
    --lambda_gan 0.5 --lambda_idt 1 --lambda_cycle 1

Process is terminated.


Inference

In [ ]:
%%bash
python src/inference_unpaired.py --model_path "output/cyclegan_turbo/my_horse2zebra/checkpoints/model_1001.pkl" \ 
    --input_image "data/my_horse2zebra/test_A/n02381460_20.jpg" \
    --prompt "picture of a zebra" --direction "a2b" \
    --output_dir "outputs" --image_prep "no_resize"